In [484]:
import pandas as pd
import numpy as np
import requests as rq
import cv2 as cv
from PIL import Image


In [485]:
# car_haar_cascade = "C:\\Coding\\GitHub\\Billboard-Car-Count\\car_detection_haar.xml"

traffic_video_one = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_one.mp4"
traffic_video_two = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_two.mp4"
traffic_video_three = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_three.mp4"
traffic_video_four = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_four.mp4"

video_height = 360
video_width = 640

In [486]:
carscrossedup = 0  # keeps track of cars that crossed up
carscrosseddown = 0  # keeps track of cars that crossed down

frame_number = 0
cars_crossed_line = 0
total_cars_seen = 0

car_ids = []
car_ids_crossed = []

car_df = pd.DataFrame(index=range)

In [487]:
video_capture = cv.VideoCapture(traffic_video_four)

# The numbers in set are property identifiers 3 = width, 4 = height
# This sets video height and width to predetermined height and width
video_capture.set(3, video_width)
video_capture.set(4, video_height)

# car_cascade_classifier = cv.CascadeClassifier(car_haar_cascade)

# 0x7634706d is used to force videowriter to output mp4 if other conflicts exist
# video_result = cv.VideoWriter('video_result.mp4', 0x7634706d, 15.0, (video_width, video_height))

background_subtractor = cv.createBackgroundSubtractorKNN()


In [488]:
# video_result = cv.VideoWriter('video_result.mp4', 0x7634706d, 15.0, (video_width, video_height))

# while True:
#     return_val, captured_image = video_capture.read()

#     if (type(captured_image) == type(None)):
#         break

#     to_grayscale = cv.cvtColor(captured_image, cv.COLOR_BGR2GRAY)

#     car_detected = car_cascade_classifier.detectMultiScale(to_grayscale, 1.1, 2)

#     for (x_cord, y_cord, width, height) in car_detected:
#         cv.rectangle(captured_image, (x_cord, y_cord), (x_cord+width, y_cord+height), (0, 255, 255), 2)

#     video_result.write(captured_image)
# video_result.release()
# cv.destroyAllWindows()

In [489]:

video_result = cv.VideoWriter('video_result.mp4', 0x7634706d, 15.0, (video_width, video_height), isColor=True)

while True:
    return_val, captured_image = video_capture.read()

    if (type(captured_image) == type(None)):
        break

    # Changes image to grayscale
    gray_image = cv.cvtColor(captured_image, cv.COLOR_BGR2GRAY)

    # Sets input file, specificies no output, and sets learning rate to 0.01
    foreground_mask = background_subtractor.apply(gray_image, None, .01)

    # Slight blur to eliminate some nosie
    grayed_blured_image = cv.blur(foreground_mask, (15, 15))

    closing_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
    closing = cv.morphologyEx(grayed_blured_image, cv.MORPH_CLOSE, closing_kernel, iterations=1)

    dilation_erosion_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(4,4))
    dilation = cv.dilate(closing ,None, dilation_erosion_kernel, iterations=10)
    erosion = cv.erode(dilation, None, dilation_erosion_kernel, iterations=7)

    # Removes grays and returns black and white (binary) image
    return_val, binary_image = cv.threshold(erosion, 100, 200, cv.THRESH_BINARY)

##############

    contours, hierarchy = cv.findContours(binary_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        cv.drawContours(captured_image, [cnt], -1, (255, 0, 0), 3, lineType=cv.LINE_4)


        hull = cv.convexHull(cnt)
        epsilon = 0.1 * cv.arcLength(cnt, True)
        approx = cv.approxPolyDP(cnt, epsilon, True)
        x_cord, y_cord, width, height = cv.boundingRect(hull)
        bounding_boxes = cv.rectangle(captured_image, (x_cord, y_cord), (x_cord + width, y_cord + height), (0, 255, 0), 3)

    # Line after which to start plotting centroids of contours
    start_line_height = 240  
    start_line = cv.line(captured_image, (0, 240), (640, 180), (0, 100, 255), 3)

    # Line which if a centroid crosses, it is counted as a car
    count_line_height = 340
    count_line = cv.line(captured_image, (0, 340), (640, 280), (0, 0, 255), 5)

    # Sets minimum size for counting of contour
    minimum_contour_size = 200

    # Sets maximum contour size, necessary for early stages of video while contours are computed
    maximum_contour_size = 2000

    x_cord_centroids = np.zeros(len(contours))
    y_cord_centroids = np.zeros(len(contours))

    for i in range(len(contours)):

        if hierarchy[0, i, 3]:

            contour_area = cv.contourArea(contours[i])

            if maximum_contour_size > contour_area and contour_area > minimum_contour_size:

                current_contour = contours[i]
                centroid_moment = cv.moments(current_contour)

                contour_x_cord = int(centroid_moment['m10'] / centroid_moment['m00'])
                contour_y_cord = int(centroid_moment['m01'] / centroid_moment['m00'])

                if contour_y_cord > start_line_height:

                    x_cord_centroids[i] = contour_x_cord
                    y_cord_centroids[i] = contour_y_cord

    x_cord_centroids = x_cord_centroids[x_cord_centroids != 0]
    y_cord_centroids = y_cord_centroids[y_cord_centroids != 0]

    minimum_x_index = []
    minimum_y_index = []

    maximum_centroid_movement= 20

    if len(x_cord_centroids):

        if not car_ids:

            for i in range(len(x_cord_centroids)):

                car_ids.append(i)
                



















    # key = cv.waitKey(30)
    # if key == 27:
    #     break

    video_result.write(bounding_boxes)
video_result.release()
cv.destroyAllWindows()